# 80 / 20

In [ ]:
import pickle as p
import numpy as np
import pandas as pd

# load the above and below 80th percentile split data
with open('Pickles/80_20_split_e100.pkl', 'rb') as file:
    split80_20 = p.load(file)

In [ ]:
split80_20.keys()

In [ ]:
import pandas as pd

# load the goal information
goals = pd.read_csv('Output/GoalOverview.csv')
goals.head(3)

In [ ]:
import os
games_included = []

game_data = (r'SUCCESS_SCORES')

for dirMD in os.listdir(game_data):
        locMD = os.path.join(game_data, dirMD)
        for dirM in os.listdir(locMD):
            games_included.append(dirM)
games_included

In [ ]:
# seconds under 10 need to be formatted to string with 0x instead of x which turns into --> x0 as a string
seconds = []
for i in range(len(goals.Second)):
    if goals.Second[i] < 10:
        seconds.append('0' + str(goals.Second[i]))
    else:
        seconds.append(str(goals.Second[i]))
        
goals['Seconds2'] = seconds

In [ ]:
# create an additional time string and minute that goes up to 90 instead of 45 for each half

goals['time_string_H'] = goals['Minute'].astype(str) + '.' + goals['Seconds2']
goals['Minute2'] = goals['Minute']
goals['Minute2'].loc[goals['Half'] == 2] = goals['Minute'] + 45

goals['time_string'] = goals['Minute2'].astype(str) + '.' + goals['Seconds2'].astype(str)



# scoring_ha = which team scored

goals.loc[goals['Scoring_Team'] == goals['Home'], 'Scoring_ha'] = -3
goals.loc[goals['Scoring_Team'] == goals['Away'], 'Scoring_ha'] = -1

goals.to_excel("Output/goal_table.xlsx")
goals

In [ ]:
# filter the relevant goals (Open play)

print(type(goals.InBox))
print(goals.InBox.unique())

print(type(goals.OpenPlay))
print(goals.OpenPlay.unique())


#  516 open play goals = relevant goals
goals.loc[(goals.OpenPlay == "True") & (goals.DFL_Name.isin(games_included))]

# 450 open play goals from within the box (relevant for the 16m Success Scores)
goals.loc[(goals.OpenPlay == "True") & (goals.InBox == True) & (goals.DFL_Name.isin(games_included))].reset_index()
# reset index because i'll be loopig over the index 
goals_rel = goals[(goals.OpenPlay == "True") & (goals.DFL_Name.isin(games_included))].reset_index()
goals_rel.drop(['index', 'Unnamed: 0'], axis=1, inplace = True)

goals_rel.to_excel('Output/Goal_Table.xlsx')

# create column to store the dictionary frame related to the goal, to check which goals are missing a frame
goals_rel['dict_entry_16m_100s'] = 'Missing'
goals_rel['dict_entry_16m_300s'] = 'Missing'
goals_rel['dict_entry_16m_500s'] = 'Missing'
goals_rel['dict_entry_30m_100s'] = 'Missing'
goals_rel['dict_entry_30m_300s'] = 'Missing'
goals_rel['dict_entry_30m_500s'] = 'Missing'

goals_rel

## Mean success score for goals and no goals
### We take all 516 open play goals as goals for 30m intervals and only the 450 open play within the box for 16m

In [ ]:
from statistics import mean
import pandas as pd
import pickle as p

# load complete data
with open('Pickles/AllDatapoints_e100.pkl', 'rb') as file:
    full_dict = p.load(file)

datasets = list(full_dict.keys())                            # datasets
goal_dict = {}                                               # dictionary for the split of succcess scores in goal and no goal
df_SS_goal = pd.DataFrame(index=['Goal', 'No_goal'])         # df for means
# thought about looking for the fitting data for each goal 
# decided to keep it as it is looking whether a goal matches the data
# have to check all of the data anyway
for dataset in datasets:
    goal_dict["_".join([dataset, 'GOAL'])] = []                  # for every previous dataset in my dictionary i now want two: 
    goal_dict["_".join([dataset, 'NOgoal'])] = [] 
    for frame in range(len(full_dict[dataset])):                # for every frame
        if dataset.startswith('30'):                                                # for the 30m intervals we don't need to check whether the goal was scored from within the penalty area
            if full_dict[dataset][frame][6] in goals_rel.time_string.values:        # to identify goals the time has to match
                index_list = np.where(goals_rel["time_string"].values == full_dict[dataset][frame][6])      # all those which match by the time stamp
                for i in index_list[0]:
                    if goals_rel.DFL_Name[i] in full_dict[dataset][frame][2] and goals_rel.Scoring_ha[i] == full_dict[dataset][frame][1] and goals_rel.Half[i] ==full_dict[dataset][frame][5]:       # have to also match by the match id and the team (scoring team) and the half (46 = HT1 vs 46 = HT2 min 1)
                        goal_dict["_".join([dataset, 'GOAL'])].append(full_dict[dataset][frame][0])         # if yes we append to the goal data set
                        if dataset == '30m_100s':    # for now i just store the goal related frame only for the 16m_100s seocond data
                            goals_rel.dict_entry_30m_100s[i] = full_dict[dataset][frame]
                        elif dataset == '30m_300s':
                            goals_rel.dict_entry_30m_300s[i] = full_dict[dataset][frame]
                        elif dataset == '30m_500s':
                            goals_rel.dict_entry_30m_500s[i] = full_dict[dataset][frame]
                        # print(split80_20[dataset][frame])
                        # print(dataset)
                    else:
                        goal_dict["_".join([dataset, 'NOgoal'])].append(full_dict[dataset][frame][0])      # else we append to the no goal data set
            else: 
                goal_dict["_".join([dataset, 'NOgoal'])].append(full_dict[dataset][frame][0])              # else we append to the no goal data set
        elif dataset.startswith('16'):                                               # for 16m intervals we have to check if scored from within the penalty are
            if full_dict[dataset][frame][6] in goals_rel.time_string[goals_rel.InBox==True].values:        # to identify goals the time has to match (==True --> from within the box)
                #print(full_dict[dataset][frame])
                index_list = np.where(goals_rel["time_string"].values == full_dict[dataset][frame][6])      # all those which match by the time stamp
                #print(index_list)
                for i in index_list[0]:
                    if goals_rel.DFL_Name[i] in full_dict[dataset][frame][2] and goals_rel.Scoring_ha[i] == full_dict[dataset][frame][1] and goals_rel.Half[i] ==full_dict[dataset][frame][5] and goals_rel.InBox[i]==True:       # have to also match by the match id and the team (scoring team) and the half and check inbox in case of two indeces in index list
                        goal_dict["_".join([dataset, 'GOAL'])].append(full_dict[dataset][frame][0])         # if yes we append to the goal data set
                        if dataset == '16m_100s':    # for now i just store the goal related frame only for the 16m_100s seocond data
                            goals_rel.dict_entry_16m_100s[i] = full_dict[dataset][frame]
                        elif dataset == '16m_300s':
                            goals_rel.dict_entry_16m_300s[i] = full_dict[dataset][frame]
                        elif dataset == '16m_500s':
                            goals_rel.dict_entry_16m_500s[i] = full_dict[dataset][frame]
                            
                            

                        # print(split80_20[dataset][frame])
                        # print(dataset)
                    else:
                        goal_dict["_".join([dataset, 'NOgoal'])].append(full_dict[dataset][frame][0])      # else we append to the no goal data set
            else: 
                goal_dict["_".join([dataset, 'NOgoal'])].append(full_dict[dataset][frame][0])              # else we append to the no goal data set

    print(dataset)
    print(mean(goal_dict["_".join([dataset, 'GOAL'])]))
    print(mean(goal_dict["_".join([dataset, 'NOgoal'])]))
    df_SS_goal[dataset] = [mean(goal_dict["_".join([dataset, 'GOAL'])]), mean(goal_dict["_".join([dataset, 'NOgoal'])])]
    df_SS_goal
    with open('Pickles/Goal_Dict_e100.pkl', 'wb') as cdict:
        p.dump(goal_dict, cdict)


In [ ]:
goal_dict['16m_100s_GOAL']

In [ ]:
goal_dict['16m_300s_GOAL']

In [ ]:
goal_dict['30m_100s_GOAL']

In [ ]:
goal_dict['30m_300s_GOAL']

In [ ]:
print(len(goal_dict['16m_100s_GOAL']))
print(len(goal_dict['16m_100s_NOgoal']))
print(len(goal_dict['16m_100s_GOAL']) + len(goal_dict['16m_100s_NOgoal']))
print(len(full_dict['16m_100s']))

In [ ]:
print(len(goal_dict['30m_100s_GOAL']))
print(len(goal_dict['30m_100s_NOgoal']))
print(len(goal_dict['30m_100s_GOAL']) + len(goal_dict['16m_100s_NOgoal']))
print(len(full_dict['30m_100s']))

In [ ]:
print(len(goal_dict['16m_300s_GOAL']))
print(len(goal_dict['16m_300s_NOgoal']))
print(len(goal_dict['16m_300s_GOAL']) + len(goal_dict['16m_100s_NOgoal']))
print(len(full_dict['16m_300s']))

In [ ]:
print(len(goal_dict['30m_300s_GOAL']))
print(len(goal_dict['30m_300s_NOgoal']))
print(len(goal_dict['30m_300s_GOAL']) + len(goal_dict['16m_100s_NOgoal']))
print(len(full_dict['30m_300s']))

In [ ]:
df_SS_goal.to_csv('Results/Success_Score_goal_e100.csv')
df_SS_goal

In [ ]:
df_SS_perc = pd.DataFrame(index=['Goal', 'No_goal'])
for dataset in datasets:
    perc_mean = []
    
    for v in range(0,2):
        perc = []
        i = 0
        for line in full_dict[dataset]:
            i += 1
            if line[0] > (df_SS_goal[dataset][v]-0.1) and line[0] < (df_SS_goal[dataset][v]+0.1):
                print(dataset)
                print(line)
                print(len(full_dict[dataset]))
                print(i/len(full_dict[dataset]))
                print(i)
                print('END OF THIS BLOCK')
                print('')
                perc.append(i/len(full_dict[dataset]))
        perc_mean.append(mean(perc))
    df_SS_perc[dataset] = perc_mean
    

In [ ]:
df_SS_perc.to_csv('Results/Success_Score_percentile_e100.csv')
df_SS_perc

In [ ]:
#need a column to which subset each frame /SS belongs
# so that i can then do what MS did and split goals scored on a frequent SS in different groups

import pandas as pd

with open('Pickles/AllDatapoints_e100.pkl', 'rb') as file:            # import the dictionary where data is not split by group or 80 percentile
    full_dict = p.load(file)

with open('Pickles/index_limits_80_e100.pkl', 'rb') as file:                 # import the boarders /limits (index) for over under 80
    limits_80 = p.load(file)

with open('Pickles/index_limits_8_e100.pkl', 'rb') as file:                 # import the boarders /limits (index) for the 8 groups
    limits_8 = p.load(file)
    
success_scores = {}                                      # create a dictionary to sort the success scores

datasets = list(full_dict.keys())                    # for all datasets by area & interval (not split)  
for dataset in datasets:
    success_scores[dataset] = []                                   # create a success score vector 
    for frame in range(len(full_dict[dataset])):               # for every frame (by number, not the frame itself)     
        success_scores[dataset].append(full_dict[dataset][frame][0])     # append the success score to the vector
     
    
# by index determine wether a success score falls in under or over the 80th percentile

for dataset in datasets:
    success_scores["_".join([dataset, 'ou'])] = np.repeat("NaN", limits_80[dataset][1])
    success_scores["_".join([dataset, 'ou'])][:limits_80[dataset][0]] = 'U'    # up to the limit it is under = U
    success_scores["_".join([dataset, 'ou'])][limits_80[dataset][0]:] = 'O'    # Upwards of the limit it is over = O


## by index determine the group a success score falls into (1-8)

for dataset in datasets:
    success_scores["_".join([dataset, "8"])] = np.repeat("NaN", limits_80[dataset][1])     # first create a vector with NaN (full length)
    for i in range(len(limits_8[dataset])):                                               # fill it based on the read limits (s.above)
        if i == 0:
            success_scores["_".join([dataset, "8"])][0:int(limits_8[dataset][i])] = i + 1    # between 0 and boarder of group 1
        else:
            success_scores["_".join([dataset, "8"])][int(limits_8[dataset][i-1]):int(limits_8[dataset][i])] = i + 1  # between boarder of group -1 and group
            


In [ ]:
success_scores.keys()

In [ ]:
success_scores['16m_100s_ou']

In [ ]:
success_scores['16m_100s_8']

In [ ]:
# sorted (ascending)
success_scores['16m_100s']

# Goals over and under 80th percentile

In [ ]:
goal_count = {}                                 # new dictionary to count the goals
datasets = list(full_dict.keys())           # datasets in this case not split based on percentiles (area & interval)

with open('Pickles/Boarder_values_10_e100.pkl', 'rb') as file:             # boarder values for the 80th percentile (the 8th element in each array)
    percentiles_10 = p.load(file)
    

for dataset in datasets:
    goal_count["_".join([dataset, 'over80'])] = 0                    # for every dataset i count goals over
    goal_count["_".join([dataset, 'under80'])] = 0                   # and under --> 2 lists in dictionary
    for frame in range(len(full_dict[dataset])):                   # every frame is checked
        if dataset.startswith('30'):
            if full_dict[dataset][frame][6] in goals_rel.time_string.values:        # to identify goals the time has to match
                index_list = np.where(goals_rel["time_string"].values == full_dict[dataset][frame][6])      # all those which match by the time stamp
                for i in index_list[0]:
                    if goals_rel.DFL_Name[i] in full_dict[dataset][frame][2] and goals_rel.Scoring_ha[i] == full_dict[dataset][frame][1] and goals_rel.Half[i] ==full_dict[dataset][frame][5]:       # have to also match by the match id and the team (scoring team) and the half (46 = HT1 vs 46 = HT2 min 1)
                        if full_dict[dataset][frame][0] < percentiles_10[dataset][8]:                                               # if under the boarder value
                            goal_count["_".join([dataset, 'under80'])] = float(goal_count["_".join([dataset, 'under80'])]) + 1 # we add + 1 to under count
                        elif full_dict[dataset][frame][0] > percentiles_10[dataset][8]:                                             # if over the boarder value
                            goal_count["_".join([dataset, 'over80'])] = float(goal_count["_".join([dataset, 'over80'])]) + 1   # we add +1  to over count
                        elif full_dict[dataset][frame][0] == percentiles_10[dataset][8]:                                            # if excatly the boarder value
                            tot_frequency = len(np.where(np.array(success_scores[dataset]) == full_dict[dataset][frame][0])[0])         # freqeuncy of frames with that success score
                            over_frequency = len(np.where((np.array(success_scores[dataset]) == full_dict[dataset][frame][0]) & (np.array(success_scores["_".join([dataset, "ou"])]) == 'O'))[0])         # freqeuncy of frames with that success score in over
                            under_frequency = len(np.where((np.array(success_scores[dataset]) == full_dict[dataset][frame][0]) & (np.array(success_scores["_".join([dataset, "ou"])]) == 'U'))[0])        # freqeuncy of frames with that success score in under
                            o_goals = over_frequency / tot_frequency                                                              # goal share for over 
                            u_goals = under_frequency / tot_frequency                                                             # goal share for under
                            goal_count["_".join([dataset, 'under80'])] = float(goal_count["_".join([dataset, 'under80'])]) + u_goals    # add the share
                            #print(goal_count["_".join([dataset, 'under80'])])
                            goal_count["_".join([dataset, 'over80'])] = float(goal_count["_".join([dataset, 'over80'])]) + o_goals      # add the share
        elif dataset.startswith('16'):
            if full_dict[dataset][frame][6] in goals_rel.time_string[goals_rel.InBox==True].values:        # to identify goals the time has to match (==True --> from within the box)
                #print(full_dict[dataset][frame])
                index_list = np.where(goals_rel["time_string"].values == full_dict[dataset][frame][6])      # all those which match by the time stamp
                #print(index_list)
                for i in index_list[0]:
                    if goals_rel.DFL_Name[i] in full_dict[dataset][frame][2] and goals_rel.Scoring_ha[i] == full_dict[dataset][frame][1] and goals_rel.Half[i] ==full_dict[dataset][frame][5] and goals_rel.InBox[i]==True:       # have to also match by the match id and the team (scoring team) and the half and check inbox in case of two indeces in index list
                        if full_dict[dataset][frame][0] < percentiles_10[dataset][8]:                                               # if under the boarder value
                            goal_count["_".join([dataset, 'under80'])] = float(goal_count["_".join([dataset, 'under80'])]) + 1 # we add + 1 to under count
                        elif full_dict[dataset][frame][0] > percentiles_10[dataset][8]:                                             # if over the boarder value
                            goal_count["_".join([dataset, 'over80'])] = float(goal_count["_".join([dataset, 'over80'])]) + 1   # we add +1  to over count
                        elif full_dict[dataset][frame][0] == percentiles_10[dataset][8]:                                            # if excatly the boarder value
                            tot_frequency = len(np.where(np.array(success_scores[dataset]) == full_dict[dataset][frame][0])[0])         # freqeuncy of frames with that success score
                            over_frequency = len(np.where((np.array(success_scores[dataset]) == full_dict[dataset][frame][0]) & (np.array(success_scores["_".join([dataset, "ou"])]) == 'O'))[0])         # freqeuncy of frames with that success score in over
                            under_frequency = len(np.where((np.array(success_scores[dataset]) == full_dict[dataset][frame][0]) & (np.array(success_scores["_".join([dataset, "ou"])]) == 'U'))[0])        # freqeuncy of frames with that success score in under
                            o_goals = over_frequency / tot_frequency                                                              # goal share for over 
                            u_goals = under_frequency / tot_frequency                                                             # goal share for under
                            goal_count["_".join([dataset, 'under80'])] = float(goal_count["_".join([dataset, 'under80'])]) + u_goals    # add the share
                            #print(goal_count["_".join([dataset, 'under80'])])
                            goal_count["_".join([dataset, 'over80'])] = float(goal_count["_".join([dataset, 'over80'])]) + o_goals      # add the share

                        
# create a dataframe storing the count of goals in a practical format and save to csv
goal_count_80_df = pd.DataFrame() 


stri = ['over80', 'under80']
strin = ['over', 'under']
for dataset in datasets:
    counts = []
    for i in range(len(stri)):
        counts.append(goal_count["_".join([dataset, stri[i]])])
        counts.append(len(split80_20["_".join([dataset, strin[i]])]))        # add the length of the data for chi square
    counts.append(len(success_scores[dataset]))
    goal_count_80_df[dataset] = counts
    


goal_count_80_df.to_csv('Results/goal_Count_80_e100.csv')
goal_count_80_df

# goals over
# data points over
# goals under
# data points under
# all datapoints

# Goals in 8 groups

In [ ]:
# load the 8 groups split data
with open('Pickles/8_split_e100.pkl', 'rb') as file:
    split_8 = p.load(file)

In [ ]:
split_8.keys()

In [ ]:
limits_8

In [ ]:
goal_count_8 = {}
datasets = list(full_dict.keys())


with open('Pickles/Boarder_values_8_e100.pkl', 'rb') as file:
    boarder_values_8 = p.load(file)
    
groups =list(range(1,9))

tot_frequency_dict = {}
group_frequencies_dict = {}
goal_share = {}

for dataset in datasets:                                   # for each combination of area & interval
    for group in groups:                                   # for each group (by percentile) --> 1 to 8
        goal_count_8["_".join([dataset, str(group)])] = 0  # empty list in the dictionary for every group for every dataset
        tot_frequency_dict["_".join([dataset, str(group)])] = []         # dictionary for the total frequency of that success score
        group_frequencies_dict["_".join([dataset, str(group)])] = []     # dictionary for the freqeuncy of that success score in that group
        goal_share["_".join([dataset, str(group)])] = []                 # dictionary for the share of the goals scored at that success score for each group
    for frame in range(len(full_dict[dataset])):       # for every frame (loop by number; not the frame itself)
        if (full_dict[dataset][frame][6] in goals_rel.time_string.values and dataset.startswith('30')) or (full_dict[dataset][frame][6] in goals_rel.time_string[goals_rel.InBox==True].values):             # is the time in the frame in the list of goals (-->potential goal frame)
            # either a 30m datapoint with the right time stamp or a 16m datapoint with the right time stamp matching only the ones in the box
            index_list = np.where(goals_rel["time_string"].values == full_dict[dataset][frame][6])     # create a list the index of the goal in the goal data frame
            for i in index_list[0]:                                                                # for every goal with that time
                if goals_rel.DFL_Name[i] in full_dict[dataset][frame][2] and goals_rel.Scoring_ha[i] == full_dict[dataset][frame][1] and goals_rel.Half[i] ==full_dict[dataset][frame][5] and (goals_rel.InBox[i]==True or dataset.startswith('30')):  # is the game name identical with the frame and the scoring team the correct one, correct half and in the box if not 30m dataset
                    for group in groups:                   # for each group (by percentile) --> 1 to 8
                        if group == 1:                                                   # if group is = 1 group-1 is not usable for lower limit so we here have to use 0 - group1 
                            if full_dict[dataset][frame][0] < boarder_values_8[dataset][group-1]:                   # is the success score smaller than the limit of group1
                                goal_count_8["_".join([dataset, str(group)])] = goal_count_8["_".join([dataset, str(group)])] + 1    # if yes it counts as a full goal to goals in group 
                            elif full_dict[dataset][frame][0] == boarder_values_8[dataset][group-1]:   # if not we check if it is exactly the boarder value; if yes we create a share based on the frequency of that success score in each group
                                tot_frequency_dict["_".join([dataset, str(group)])] = len(np.where(np.array(success_scores[dataset]) == full_dict[dataset][frame][0])[0])   # the tot_frequency is frequency of that success score across all groups
                                #print(tot_frequency_dict["_".join([dataset, str(group)])])
                                group_frequencies_dict["_".join([dataset, str(group)])] = len(np.where((np.array(success_scores[dataset]) == full_dict[dataset][frame][0]) & (np.array(success_scores["_".join([dataset, "8"])]) == str(group)))[0])  # the freqeuncy of that success score in this group
                                #print(group_frequencies_dict["_".join([dataset, str(group)])])
                                goal_share["_".join([dataset, str(group)])] = group_frequencies_dict["_".join([dataset, str(group)])]/tot_frequency_dict["_".join([dataset, str(group)])]   # goal share = frequency in that group / overall frequency (* goals with that success score)
                                #print(goal_count_8["_".join([dataset, str(group)])])
                                goal_count_8["_".join([dataset, str(group)])] = float(goal_count_8["_".join([dataset, str(group)])]) + goal_share["_".join([dataset, str(group)])]      # finally we add that share to that group
                                #print(goal_share["_".join([dataset, str(group)])])
                                #print(goal_count_8["_".join([dataset, str(group)])])
                        else:
                            if full_dict[dataset][frame][0] < boarder_values_8[dataset][group-1] and full_dict[dataset][frame][0] > boarder_values_8[dataset][group-2]: # if group is > 1 we can just check with limits (group -1 upper limit to group upper limit)
                                goal_count_8["_".join([dataset, str(group)])] = goal_count_8["_".join([dataset, str(group)])] + 1                   # if yes it counts as a full goal to goals in group 
                            elif (full_dict[dataset][frame][0] == boarder_values_8[dataset][group-1]) | (full_dict[dataset][frame][0] == boarder_values_8[dataset][group-2]):                      # if not we check if it is exactly the boarder value; if yes we create a share based on the frequency of that success score in each group
                                tot_frequency_dict["_".join([dataset, str(group)])] = len(np.where(np.array(success_scores[dataset]) == full_dict[dataset][frame][0])[0])                           # the tot_frequency is frequency of that success score across all groups
                                group_frequencies_dict["_".join([dataset, str(group)])] = len(np.where((np.array(success_scores[dataset]) == full_dict[dataset][frame][0]) & (success_scores["_".join([dataset, "8"])] == str(group)))[0]) # the tot_frequency is frequency of that success score across all groups
                                goal_share["_".join([dataset, str(group)])]= group_frequencies_dict["_".join([dataset, str(group)])]/tot_frequency_dict["_".join([dataset, str(group)])]           # goal share = frequency in that group / overall frequency (* goals with that success score)
                                goal_count_8["_".join([dataset, str(group)])] = float(goal_count_8["_".join([dataset, str(group)])]) + goal_share["_".join([dataset, str(group)])]                # finally we add that share to that group                                   
                                #print(goal_count_8["_".join([dataset, str(group)])])

                                
# create a dataframe storing the count of goals in a practical format and save to csv
goal_count_8_df = pd.DataFrame() 

for dataset in datasets:
    counts = []
    for i in list(range(1,9)):
        counts.append(goal_count_8["_".join([dataset, str(i)])])
    goal_count_8_df[dataset] = counts
    


goal_count_8_df.to_csv('Results/goal_Count_8_e100.csv')
goal_count_8_df